In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from scipy import stats, signal
from scipy.signal import butter
import scipy
from pyedflib import highlevel
from path import Path
import os

# Reading in data from edf files

In [ ]:
# Drive data link: 
# EE481: https://drive.google.com/drive/folders/1vh3GeBGvYB9J9DZ8yG7Fvvvbqj-QnXb2
# EE464: https://drive.google.com/drive/folders/1wVDt9dwh_dQE2izJJMP19tKpQne9xzr3
# EE461: https://drive.google.com/drive/folders/1-BFvnQYDbcQW_0UnB_lVnqPhoGRh2aeA

dpath = ['../re-synchornize/EP2_PSG_EE481_EB_DEV6751_1621093397.edf' 
         ,'../re-synchornize/EP2_PSG_EE464_EB_DEV755b_1620834065.edf' 
         ,'../re-synchornize/EP2_PSG_EE461_EB_DEV6751_1620745140.edf']


subject_list = [str(Path(dpath[i]).name).split('_')[2] for i in range(len(dpath))]
print(subject_list)

channel_indices = [22,23,24] # SpO2, ECG I, ECG II
signals, signal_headers, _ = highlevel.read_edf(dpath[0],ch_nrs=channel_indices)
signal_name = []
srate_values = []
winlength = [10,15,30] #10, 15, and 30 seconds window
stride = [10,15,30]

for i in range(len(signals)):
    signal_name.append(signal_headers[i]['label'])
    srate_values.append(signal_headers[i]['sample_rate'])

# SpO2 stride, SPO2 array of moving window, and window length of both HR and SPO2

In [ ]:
# SPO2 stride. 1D list. [10*10, 15*10, 30*10]
spo2_stride_list = [stride[i]*srate_values[0] for i in range(len(stride))]
    
# Winlength for both HR and SPO2. 2D list.
# row: different winlength | col: difference in sampling rate
winlength_list = [[winlength[0]*srate_values[i] for i in range(len(winlength))],
                  [winlength[1]*srate_values[i] for i in range(len(winlength))],
                  [winlength[2]*srate_values[i] for i in range(len(winlength))]]

# SPO2 winonset. 2D list. row: different winlength | col: an array of corresponding winlength
SPO2_winonset = [np.arange(0,len(signals[0])-winlength_list[i][0],spo2_stride_list[i]).tolist() for i in range(3)]

In [ ]:
# Calculating SPO2 with 3 different window lengths
spo2_mean = []
for j in range(len(SPO2_winonset)):
    spo2_mean.append([])
    for i in range(len(SPO2_winonset[j])):
        data_pack = signals[0][int(SPO2_winonset[j][i]):int(SPO2_winonset[j][i]+winlength[0])]
        spo2_mean[j].append(np.mean(data_pack))
    
# Creating spo2_file list that contains time and data
spo2_file = []
for j in range(len(spo2_mean)):
    spo2_file.append([])
    for i in range(len(spo2_mean[j])):
        spo2_file[j].append([winonset_list[j][i]/srate_values[0],spo2_mean[j][i]])
    
# Saving spo2 data into csv files
for name in subject_list:
    os.mkdir(name)
    for i, win in enumerate(winlength):
        fpath = os.path.join(name,name+'_spo2_label_'+str(win)+'.csv')
        np.savetxt(fpath, spo2_file[i], delimiter=",", fmt='%10.2f')

# np.savetxt("EE461/EE461_spo2_label_10s.csv", spo2_file[0], delimiter=",", fmt='%10.2f')
# np.savetxt("EE461/EE461_spo2_label_15s.csv", spo2_file[1], delimiter=",", fmt='%10.2f')
# np.savetxt("EE461/EE461_spo2_label_30s.csv", spo2_file[2], delimiter=",", fmt='%10.2f')

# Heart rate stride and array of moving windows (HR_winonset)

In [ ]:
# Filtered ECG I signal | sampling rate 200Hz
upper_f = 3
lower_f = 1
filt_sig = butter_bandpass_filter(signals[1], lower_f, upper_f, srate_values[1], order=1)

# Stride with 10s, 15s, and 30s
HR_stride =  [stride[i]*srate_values[1] for i in range(len(stride))]

# winonset for heart rate with 10s, 15s, and 30s windows
HR_winonset = [np.arange(0,len(signals[1])-winlength_list[i][1],HR_stride[i]).tolist() for i in range(3)]

In [ ]:
# Calculating Heart Rate
hrate = []
for i in range(len(HR_winonset)):
    hrate.append([])
    for win in range(0,len(HR_winonset[i])):
        heart_rate, heart_rate_std, _ = compute_heart_rate(sig[HR_winonset[i][win]:HR_winonset[i][win]+time],200)
        hrate[i].append(heart_rate)
        
# Saving heart rate into csv files
hrate_file = []
for i in range(len(hrate)):
    hrate_file.append([])
    for j in range(len(hrate[i])):
        hrate_file[i].append([winonset_list[i][j],hrate[i][j]])
        
# Saving Heart rate data into csv files
np.savetxt('EE461/EE461_HR_label_10s.csv', hrate_file[0], delimiter=',', fmt='%10.1f')
np.savetxt('EE461/EE461_HR_label_15s.csv', hrate_file[1], delimiter=',', fmt='%10.1f')
np.savetxt('EE461/EE461_HR_label_30s.csv', hrate_file[2], delimiter=',', fmt='%10.1f')

# Functions to find peaks

In [ ]:
def find_peaks(x, scale=None, debug=False):
    """
    https://github.com/ig248/pyampd/blob/master/pyampd/ampd.py
    
    Find peaks in quasi-periodic noisy signals using AMPD algorithm.
    Extended implementation handles peaks near start/end of the signal.
    Optimized implementation by Igor Gotlibovych, 2018
    @input:
        x : ndarray
            1-D array on which to find peaks
        scale : int, optional
            specify maximum scale window size of (2 * scale + 1)
        debug : bool, optional
            if set to True, return the Local Scalogram Matrix, `LSM`,
            weigted number of maxima, 'G',
            and scale at which G is maximized, `l`,
            together with peak locations
    @output:
        pks: ndarray
            The ordered array of peak indices found in `x`
    """
    x = signal.detrend(x)
    N = len(x)
    L = N // 2
    if scale:
        L = min(scale, L)

    # create LSM matix
    LSM = np.ones((L, N), dtype=bool)
    for k in np.arange(1, L + 1):
        LSM[k - 1, 0:N - k] &= (x[0:N - k] > x[k:N])  # compare to right neighbours
        LSM[k - 1, k:N]     &= (x[k:N] > x[0:N - k])  # compare to left neighbours

    # Find scale with most maxima
    G = LSM.sum(axis=1)
    G = G * np.arange( N // 2, N // 2 - L, -1)  # normalize to adjust for new edge regions
    l_scale = np.argmax(G)

    # find peaks that persist on all scales up to l
    pks_logical = np.min(LSM[0:l_scale, :], axis=0)
    pks = np.flatnonzero(pks_logical)
    if debug:
        return pks, LSM, G, l_scale
    return pks

def compute_heart_rate(data, srate):
    """
    Compute heart rate from data
    @input: 
            data (ndarray): 1-D array on which to compute the hear rate
            srate (float): sampling rate of the data in Hz

    @output: 
            heart_beat_mean (float): mean heart beats per minute in the data segment
            heart_beat_std (float): standart deviation of the computed hear beat in the data segment
            peaks (ndarray): indices of peak point in data
    """
    peaks = find_peaks(data)
    
    # remove the first and last peak
    peaks = peaks[1:-1] 
    
    # compute the interval between beats
    beat_intervals = np.array([ (peaks[i] - peaks[i-1])/(srate*60) for i in range(1,len(peaks))])
    
    # compute the beats per minute
    heart_beats = 1/beat_intervals
    heart_beat_mean = np.mean(heart_beats)
    heart_beat_std = np.std(heart_beats)

    return heart_beat_mean, heart_beat_std, peaks

def butter_bandpass_filter(data, lowcut, highcut, fs, order=1):
    b, a = butter(order, [lowcut, highcut], btype='band',fs=fs)
    y = signal.filtfilt(b, a, data)
    return y